<a href="https://colab.research.google.com/github/komh9575/first-repository/blob/main/%20Feature%20Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

In [5]:
dating_df = pd.read_csv('/content/sample_data/dating (1).csv')


In [6]:
print(dating_df)

      gender   age  age_o                                  race  \
0     female  21.0   27.0  Asian/PacificIslander/Asian-American   
1     female  21.0   22.0  Asian/PacificIslander/Asian-American   
2     female  21.0   22.0  Asian/PacificIslander/Asian-American   
3     female  21.0   23.0  Asian/PacificIslander/Asian-American   
4     female  21.0   24.0  Asian/PacificIslander/Asian-American   
...      ...   ...    ...                                   ...   
8373    male  25.0   26.0           European/Caucasian-American   
8374    male  25.0   24.0           European/Caucasian-American   
8375    male  25.0   29.0           European/Caucasian-American   
8376    male  25.0   22.0           European/Caucasian-American   
8377    male  25.0   22.0           European/Caucasian-American   

                                    race_o  importance_same_race  \
0              European/Caucasian-American                   2.0   
1              European/Caucasian-American                 

In [18]:
rename_map = {
    "pref_o_attractive": "o_important_attractive",
    "attractive_o": "o_score_attractive",
    "attractive_important": "i_important_attractive",
    "attractive_partner": "i_score_attractive",
    "pref_o_sincere": "o_important_sincere",
    "sincere_partner": "i_score_sincere",
    "pref_o_intelligence": "o_important_intelligence",
    "intelligence_partner": "i_score_intelligence",
    "pref_o_funny": "o_important_funny",
    "funny_partner": "i_score_funny",
    "pref_o_ambition": "o_important_ambition",
    "ambition_partner": "i_score_ambition",
    "pref_o_shared_interests": "o_important_shared_interests",
    "shared_interests_partner": "i_score_shared_interests"
}
dating_df = dating_df.rename(columns=rename_map)
print(dating_df)

      gender   age  age_o                                  race  \
0     female  21.0   27.0  Asian/PacificIslander/Asian-American   
1     female  21.0   22.0  Asian/PacificIslander/Asian-American   
2     female  21.0   22.0  Asian/PacificIslander/Asian-American   
3     female  21.0   23.0  Asian/PacificIslander/Asian-American   
4     female  21.0   24.0  Asian/PacificIslander/Asian-American   
...      ...   ...    ...                                   ...   
8373    male  25.0   26.0           European/Caucasian-American   
8374    male  25.0   24.0           European/Caucasian-American   
8375    male  25.0   29.0           European/Caucasian-American   
8376    male  25.0   22.0           European/Caucasian-American   
8377    male  25.0   22.0           European/Caucasian-American   

                                    race_o  importance_same_race  \
0              European/Caucasian-American                   2.0   
1              European/Caucasian-American                 

In [13]:
drop_cols = []

for i in dating_df.columns:
    if i.startswith('o_important_') or i.startswith('i_important_'):
        drop_cols.append(i)

# The original intent for `dating_df.columns = new_cols` is unclear given the previous logic.
# If the goal was to drop columns, the next step would be to use drop_cols.
# For now, I'm commenting out the line that would cause a NameError and just building the drop_cols list correctly.
# If you meant to drop these columns, the next step would be:
# dating_df = dating_df.drop(columns=drop_cols)

In [14]:
# drop_cols에 속해있는 변수들의 결측치 행들을 제거하고 dating_df로 저장
dating_df = dating_df.dropna(subset = drop_cols)
dating_df = dating_df.fillna(-99)

In [ ]:
# "o_score_", "i_score_"로 시작되는 변수 중, 10점 이상인 값은 10으로 대체(lambda 함수 활용하기)
for col in dating_df.columns:
    if col.startswith('o_score_') or col.startswith('i_score_'):
        dating_df[col] = dating_df[col].apply(lambda x: 10 if x > 10 else x)

In [20]:
# "o_important_", "i_important"로 시작되는 각각의 6항목의 합은 100이 되어야한다.
# 합이 100이 아닌 행들을 찾아 가중치를 부여하여 합이 100이 되도록 만들어보자.

# o_important로 시작하는 변수 모으기
o_imp = []

for col in dating_df.columns:
    if col.startswith('o_important_'):
        o_imp.append(col)

# o_important로 시작하는 변수들의 합 구하기
dating_df['o_imp_sum'] = dating_df[o_imp].sum(axis=1)

# i_important로 시작하는 변수 모으기
i_imp = []

for col in dating_df.columns:
    if col.startswith('i_important_'):
        i_imp.append(col)

dating_df['i_imp_sum'] = dating_df[i_imp].sum(axis=1)

# 가중치 부여하여 업데이트하기
dating_df[o_imp] = dating_df.apply(lambda x: (100 / x['o_imp_sum']) * x[o_imp] if x['o_imp_sum'] != 0 else x[o_imp], axis=1)
dating_df[i_imp] = dating_df.apply(lambda x: (100 / x['i_imp_sum']) * x[i_imp] if x['i_imp_sum'] != 0 else x[i_imp], axis=1)

# 불필요 컬럼 제거
dating_df.drop(['o_imp_sum','i_imp_sum'], axis=1, inplace = True)

In [25]:
# "남자의 나이-여자의 나이" 를 구하는 함수를 age_func 이름으로 만들기 (나이에 -99가 있는 경우는 -99를 return)
def age_func(x):
    if x['age'] == -99 or x['age_o'] == -99:
        return -99
    elif x['gender']=='female':
        return x['age_o'] - x['age']
    else:
        return x['age'] - x['age_o']

In [27]:
# dating_df를 age_func 함수에 적용시키고, 결과를 "age_gap" 이름의 새 변수로 저장
dating_df['age_gap'] = dating_df.apply(age_func, axis=1)

In [ ]:
#elif는 lambda 안에서 지원되지 않음
#elif 'positive'처럼 조건 없이 문자열만 있어서 잘못된 구문
dating_df['age_gap_dir'] = dating_df['age_gap'].apply(
    lambda x: 'negative' if x < 0 else ('positive' if x > 0 else 'zero'))

In [33]:
print(dating_df['age_gap'] )

0       6.0
1       1.0
2       1.0
3       2.0
4       3.0
       ... 
8373   -1.0
8374    1.0
8375   -4.0
8376    3.0
8377    3.0
Name: age_gap, Length: 8210, dtype: float64


In [35]:
# "age_gap" 변수를 절대값 처리하여 같은 이름으로 저장
dating_df['age_gap'] = dating_df['age_gap'].abs()

In [38]:
# 양측 race 정보의 동일 여부를 비교하여, 동일할 경우 1, 다를 경우 -1로 "same_race"라는 새변수로 저장
dating_df['same_race'] = dating_df.apply(lambda x: 1 if x['race'] == x['race_o'] else -1, axis=1)

In [41]:
# "same_race"와 "importance_same_race" 변수를 곱하여 "same_race_point"라는 새변수로 저장
dating_df['same_race_point'] = dating_df['same_race'] * dating_df['importance_same_race']

In [43]:
# "o_important", "o_score", "i_important", "i_score"로 시작하는 변수 이름을 각각의 리스트로 저장
o_important = []
o_score = []
i_important = []
i_score = []

for i in dating_df.columns:
    if i.startswith('o_important'):
        o_important.append(i)
    elif i.startswith('o_score'):
        o_score.append(i)
    elif i.startswith('i_important'):
        i_important.append(i)
    elif i.startswith('i_score'):
        i_score.append(i)

In [44]:
# "o_important", "i_important"로 시작하는 변수에서 0인 값을 -99로 변경하여 저장
dating_df[o_important] =dating_df[o_important].replace({0: -99})
dating_df[i_important] = dating_df[i_important].replace({0: -99})

In [45]:
# rating 함수 만들기: important항목과 score 항목을 곱하되, -99가 들어있는 경우는 -99가 return되도록 정의
def rating(data, important, score):
    if data[important] == -99:
        return -99
    elif data[score] == -99:
        return -99
    else:
        return data[important] * data[score]

In [47]:
# 계산된 rating에 대한 컬럼 이름 설정
o_cols = [
    'o_rating_attractive', 'o_rating_sincere', 'o_rating_intellicence',
    'o_rating_funny', 'o_rating_ambtition', 'o_rating_shared_interests'
]

i_cols = [
    'i_rating_attractive', 'i_rating_sincere', 'i_rating_intellicence',
    'i_rating_funny', 'i_rating_ambtition', 'i_rating_shared_interests'
]


In [49]:
# rating함수를 통해 "나"의 rating 값과 "상대"의 rating 값을 각각 계산하고
# 위에 정의된 이름의 변수로 저장 (for loop와 zip을 사용)
for i, j, k in zip(o_important, o_score, o_cols):
    dating_df[k] = dating_df.apply(lambda x: rating(x, i, j), axis=1)

for i, j, k in zip(i_important, i_score, i_cols):
    dating_df[k] = dating_df.apply(lambda x: rating(x, i, j), axis=1)